# Interactive Geospatial Visualization with Kepler.gl

This notebook demonstrates how to create interactive geospatial visualizations using the `kepler.gl` library in Python. It visualizes remote sensing raster data (e.g., Sentinel-2 or Landsat RGB composites) and vector data (e.g., area of interest boundaries or detected objects) on an interactive map. This is particularly useful for exploring spatial patterns in remote sensing data dynamically.

## Prerequisites
- Install required libraries: `keplergl`, `rasterio`, `geopandas`, `numpy`, `matplotlib` (listed in `requirements.txt`).
- A GeoTIFF file (e.g., `sentinel_rgb.tif` or `landsat_rgb.tif`) from a previous notebook (e.g., `21_download_data.ipynb`).
- A GeoJSON or shapefile with vector data (e.g., `aoi.geojson` or `yolo_detections.shp` from `20_object_detection_yolo.ipynb`).
- Replace file paths with your own data.

## Learning Objectives
- Load and prepare raster and vector data for Kepler.gl visualization.
- Create an interactive map with raster and vector layers using Kepler.gl.
- Save the interactive map as an HTML file for sharing.

In [ ]:
# Import required libraries
from keplergl import KeplerGl
import rasterio
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import os
import imageio
from rasterio.warp import transform_bounds

## Step 1: Load Raster and Vector Data

Load the RGB composite GeoTIFF and vector data (e.g., AOI or detected objects).

In [ ]:
# Define file paths
raster_path = 'remote_sensing_data/sentinel_rgb.tif'  # Replace with your RGB GeoTIFF
vector_path = 'yolo_detections.shp'  # Replace with your vector file (e.g., AOI or detections)

# Load raster
with rasterio.open(raster_path) as src:
    raster_data = src.read().transpose(1, 2, 0)  # Shape: (height, width, bands)
    raster_bounds = src.bounds
    raster_crs = src.crs

# Load vector data
vector_gdf = gpd.read_file(vector_path)
if vector_gdf.crs != raster_crs:
    vector_gdf = vector_gdf.to_crs(raster_crs)

# Print basic information
print(f'Raster shape: {raster_data.shape}')
print(f'Raster CRS: {raster_crs}')
print(f'Vector CRS: {vector_gdf.crs}')
print(f'Number of vector features: {len(vector_gdf)}')

## Step 2: Prepare Raster for Kepler.gl

Normalize the RGB raster and save it as a PNG for Kepler.gl compatibility.

In [ ]:
# Normalize raster for visualization
raster_normalized = raster_data / np.percentile(raster_data, 98) if np.percentile(raster_data, 98) > 0 else raster_data
raster_normalized = np.clip(raster_normalized, 0, 1)

# Convert bounds to WGS84 (EPSG:4326) for Kepler.gl
bounds_latlon = transform_bounds(raster_crs, 'EPSG:4326', *raster_bounds)

# Save normalized raster as PNG
temp_png = 'temp_raster.png'
imageio.imwrite(temp_png, (raster_normalized * 255).astype(np.uint8))

print(f'Temporary PNG saved to: {temp_png}')

## Step 3: Prepare Vector Data for Kepler.gl

Convert vector data to WGS84 and prepare it for Kepler.gl.

In [ ]:
# Reproject vector data to WGS84
vector_gdf_wgs84 = vector_gdf.to_crs('EPSG:4326')

# Save as GeoJSON for Kepler.gl
temp_geojson = 'temp_vector.geojson'
vector_gdf_wgs84.to_file(temp_geojson, driver='GeoJSON')

print(f'Temporary GeoJSON saved to: {temp_geojson}')

## Step 4: Create Kepler.gl Map

Initialize a Kepler.gl map and add the raster and vector layers.

In [ ]:
# Initialize Kepler.gl map
map_config = {
    'version': 'v1',
    'config': {
        'mapState': {
            'latitude': (bounds_latlon[1] + bounds_latlon[3]) / 2,
            'longitude': (bounds_latlon[0] + bounds_latlon[2]) / 2,
            'zoom': 10
        }
    }
}
m = KeplerGl(height=600, config=map_config)

# Add raster as an image layer
m.add_data(
    data={'type': 'image', 'url': temp_png, 'bounds': [bounds_latlon[0], bounds_latlon[1], bounds_latlon[2], bounds_latlon[3]]},
    name='Raster Layer'
)

# Add vector data
with open(temp_geojson, 'r') as f:
    m.add_data(data=f.read(), name='Vector Layer')

# Display map
m

## Step 5: Save the Kepler.gl Map

Save the interactive map as an HTML file for sharing.

In [ ]:
# Save map to HTML
output_map_path = 'kepler_map.html'
m.save_to_html(file_name=output_map_path)

print(f'Interactive map saved to: {output_map_path}')

# Clean up temporary files
os.remove(temp_png)
os.remove(temp_geojson)

## Step 6: Static Visualization for Reference

Create a static visualization of the RGB composite with vector data for reference.

In [ ]:
# Plot static RGB composite with vector data
fig, ax = plt.subplots(figsize=(8, 8))
ax.imshow(raster_normalized)
vector_gdf.plot(ax=ax, facecolor='none', edgecolor='red', linewidth=2)
plt.title('Static RGB Composite with Vector Data')
plt.xlabel('Column')
plt.ylabel('Row')
plt.show()

## Next Steps

- Replace `sentinel_rgb.tif` with your own RGB GeoTIFF (e.g., from `21_download_data.ipynb`).
- Replace `yolo_detections.shp` with your own vector file (e.g., AOI from `aoi.geojson`).
- Customize the Kepler.gl map by adjusting the configuration (e.g., layer styles, colors) via the Kepler.gl interface.
- Explore advanced Kepler.gl features like time series animation or heatmaps.
- Revisit previous notebooks (e.g., `21_download_data.ipynb`, `22_folium_visualization.ipynb`) for data preprocessing or other visualization techniques.

## Notes
- Ensure raster and vector data have compatible CRS; Kepler.gl requires data in EPSG:4326.
- Large rasters may need downsampling to avoid memory issues in Kepler.gl.
- Use the Kepler.gl interface to fine-tune layer styles (e.g., colors, opacity) interactively.
- See `docs/installation.md` for troubleshooting library installation.